### Content of this notebook

Two regression problems: predict number of crimes per week and average number of crimes per day 
Preparation code does both. Some steps are the same where otheres are different
weekly specific variables are designated with prefix _w and daily average with _d
Regression algoritms are the same and will be executed twice by changing input 

#### Import nesessary modules

In [15]:
import numpy as np
import math as math 
import pandas as pd
import matplotlib as plt
import seaborn as sns
from matplotlib import pyplot as plt
import datetime
import calendar
# from datetime import datetime

from astral.sun import sun
from astral import LocationInfo

import warnings

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder

from shapely.geometry.point import Point
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn import metrics

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

#### Read input data

In [4]:
df_all = pd.read_csv('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/NYPD_Complaint_All_Clean.csv', sep=",")

subway = pd.read_csv('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/SUBWAY_ENTRANCE.csv')

nynta = pd.read_csv ('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/nynta.csv')

nyc_pop = pd.read_csv('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv')
nyc_pop2010 = nyc_pop[nyc_pop['Year'] == 2010]

holidays = pd.read_csv ('/Volumes/CUNYGC/GC/CIS74020_ML/Final Project/Data/US_Holiday_Dates.csv')

## Cassification problem

## KNN

In [10]:
ofns_list = ['BURGLARY', 'INTOXICATED & IMPAIRED DRIVING', 'DANGEROUS DRUGS']
# df_knn = df_all[(df_all['OFNS_DESC'].isin(ofns_list)) & (df_all['YEAR'] <= 2010)]
df_knn = df_all[df_all['OFNS_DESC'].isin(ofns_list)]

y=df_knn['OFNS_DESC']

# X=df_knn[['NTACode','YEAR', 'WEEK','DAY','DAY_OF_WEEK', 'TIME_CAT', 'INSIDE']]
X=df_knn[['NTACode','YEAR', 'WEEK','DAY','DAY_OF_WEEK', 'TIME', 'TIME_CAT', 'INSIDE']]
# X=df_knn.drop('LAW_CAT_CD', axis=1)

# X = pd.get_dummies(X, columns=['NTACode','YEAR','WEEK','DAY','DAY_OF_WEEK', 'TIME', 'TIME_CAT', 'INSIDE'])
X = pd.get_dummies(X, columns=['NTACode','YEAR','WEEK','DAY','DAY_OF_WEEK', 'TIME_CAT', 'INSIDE'])

X = np.asarray (X)
y = np.asarray (y)

In [27]:
X1= X[:20000,:]
y1= y[:20000]

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

neighbors = [3,5,10,20,50]
wghts=['uniform', 'distance']
distance=[1,2] #1-manhattan; 2-euclidean

models = {}
for i in range(len(neighbors)):       
    for j in range(len(wghts)):       
        for w in range(len(distance)): 
            knn = KNeighborsClassifier(n_neighbors=neighbors[i], weights=wghts[j],p=distance[w])
            knn.fit(X_train, y_train)       
            train_accuracy = knn.score(X_train, y_train)
            
            key = [neighbors[i], wghts[j], distance[w]]
            models[tuple(key)] = [train_accuracy]
            
            test_accuracy = knn.score(X_test, y_test)
            key = [neighbors[i], wghts[j], distance[w]]
            models[tuple(key)] = [train_accuracy, test_accuracy]
            
models_sorted=sorted(models.items(), key=lambda elem: elem[1][1], reverse=True)  
     
for i in range(len(models_sorted)):
    print('Parameters: ', models_sorted[i][0],'; Trainig accuracy: {:.3f}; test accuracy: {:.3f}'.format(models_sorted[i][1][0],models_sorted[i][1][1]) )


Parameters:  (10, 'distance', 1) ; Trainig accuracy: 0.997; test accuracy: 0.737
Parameters:  (10, 'distance', 2) ; Trainig accuracy: 0.997; test accuracy: 0.734
Parameters:  (50, 'distance', 1) ; Trainig accuracy: 0.997; test accuracy: 0.734
Parameters:  (20, 'distance', 1) ; Trainig accuracy: 0.997; test accuracy: 0.732
Parameters:  (50, 'distance', 2) ; Trainig accuracy: 0.997; test accuracy: 0.730
Parameters:  (20, 'distance', 2) ; Trainig accuracy: 0.997; test accuracy: 0.729
Parameters:  (10, 'uniform', 1) ; Trainig accuracy: 0.780; test accuracy: 0.726
Parameters:  (10, 'uniform', 2) ; Trainig accuracy: 0.780; test accuracy: 0.726
Parameters:  (5, 'distance', 1) ; Trainig accuracy: 0.997; test accuracy: 0.726
Parameters:  (5, 'distance', 2) ; Trainig accuracy: 0.997; test accuracy: 0.721
Parameters:  (20, 'uniform', 1) ; Trainig accuracy: 0.753; test accuracy: 0.719
Parameters:  (20, 'uniform', 2) ; Trainig accuracy: 0.753; test accuracy: 0.719
Parameters:  (50, 'uniform', 1) ; 

In [13]:
X1= X 
y1= y 
# X1= X[:100000,:]
# y1= y[:100000]

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

#Selected parameters of the best model from previous step
# =====> Parameters:  (10, 'distance', 1) ; Trainig accuracy: 0.997; test accuracy: 0.737

s_neighbors = 10
s_wghts='distance'
s_distance=1 # 1- manhattan

knn=KNeighborsClassifier(n_neighbors=s_neighbors, weights=s_wghts,p=s_distance)

knn.fit(X_train, y_train)
train_accuracy = knn.score(X_train, y_train)
print('Train Accuracy', train_accuracy)

test_accuracy_knn = knn.score(X_test, y_test)
print("\nTest Accuracy: ", test_accuracy_knn)

Train Accuracy 0.9961875

Test Accuracy:  0.73205


In [16]:
print("Evaluation of the KNN Model")

# The accuracy of the model
test_accuracy_knn = knn.score(X_test, y_test)
print("\nTest Accuracy: ", test_accuracy_knn)

# No. of Correct Predictions
y_test_predicted = knn.predict(X_test)
print("\nNo. of correct predictions (Test): %d/%d" % (np.sum(y_test_predicted == y_test), len(y_test)))


# Confusion Matrix
print("\nConfusion Matrix (Test Data):\n", confusion_matrix(y_test, y_test_predicted))

print('\nTrain Classification Report:')
print(classification_report(y_test, y_test_predicted))

Evaluation of the KNN Model

Test Accuracy:  0.73205

No. of correct predictions (Test): 14641/20000

Confusion Matrix (Test Data):
 [[3760 1990  171]
 [1106 9453  651]
 [  64 1377 1428]]

Train Classification Report:
                                precision    recall  f1-score   support

                      BURGLARY       0.76      0.64      0.69      5921
               DANGEROUS DRUGS       0.74      0.84      0.79     11210
INTOXICATED & IMPAIRED DRIVING       0.63      0.50      0.56      2869

                      accuracy                           0.73     20000
                     macro avg       0.71      0.66      0.68     20000
                  weighted avg       0.73      0.73      0.73     20000

